In [64]:
require 'torch'
require 'nn'
require 'rnn'

In [65]:
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

In [66]:
-- Simulating the data
qValues = torch.rand(10, 2)

-- Generating the max values and getting the indices
qMax, qindx = torch.max(qValues, 2)

--- I want to select the qindx elements for each row
actions = torch.zeros(10, 2):scatter(2, qindx, torch.ones(qValues:size()))

-- Simulating streams and queries
sentences = genNbyK(10, 15, 1, 100)
queries = genNbyK(10, 4, 1, 100)

In [76]:
sentences:size()

 10
 15
[torch.LongStorage of size 2]



In [92]:
actions:select(2, 1), sentences

 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
[torch.DoubleTensor of size 10]

  21   21   60   53   12   72   65    9   87   97   25    4   73   93    2
  44   45   21   82   47   79   22    1   28   33   30   49   30   42   98
  55   55   84   14   22   14   46    3   57   23   24   12   77   79    8
  33   33   64   83   50   57   70   91   10   52    3   49   18   49   27
  65   65   83   70   81   98   69   92   83   22  100   50   13   20    4
  26   26   93   68   91   76   96   50   84   54   80   91   91   30   81
  51   51   19   28   30   47   48   48    1   46   31   64   34   12   21
  36   23   91    9   11    6   59   28   21   32   66   85   58   57    5
  46    9   17   37   67    4    1   81   25   86   74    7   14   84   14
  52   57   41   41   49   89   55   32   13   26   26   92   40   72   40
[torch.LongTensor of size 10x15]



In [96]:
actions:byte()

 0  1
 1  0
 0  1
 0  1
 0  1
 0  1
 0  1
 1  0
 1  0
 1  0
[torch.ByteTensor of size 10x2]



In [94]:
actions:gather(1, actions:select(2, 1))

[string "local f = function() return actions:gather(1,..."]:1: invalid arguments: DoubleTensor number DoubleTensor 
expected arguments: [*DoubleTensor*] DoubleTensor index LongTensor
stack traceback:
	[C]: in function 'f'
	[string "local f = function() return actions:gather(1,..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0101b8ed20: 

In [97]:
nn.MaskedSelect:forward({actions:select(2, 1):byte(), sentences})

...ierarceo/torch/install/share/lua/5.1/nn/MaskedSelect.lua:18: attempt to index field 'output' (a nil value)
stack traceback:
	...ierarceo/torch/install/share/lua/5.1/nn/MaskedSelect.lua:18: in function 'f'
	[string "local f = function() return nn.MaskedSelect:f..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0101b8ed20: 

In [93]:
actions

 0  1
 1  0
 0  1
 0  1
 0  1
 0  1
 0  1
 1  0
 1  0
 1  0
[torch.DoubleTensor of size 10x2]



In [67]:
qValues, actions

 0.5031  0.8846
 0.7346  0.4749
 0.8112  0.9227
 0.5279  0.6577
 0.9940  0.9977
 0.3802  0.5912
 0.7998  0.8937
 0.5356  0.2557
 0.4890  0.1229
 0.8800  0.5584
[torch.DoubleTensor of size 10x2]

 0  1
 1  0
 0  1
 0  1
 0  1
 0  1
 0  1
 1  0
 1  0
 1  0
[torch.DoubleTensor of size 10x2]



In [1]:
function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [62]:
model = buildModel('bow', 100, 50, 'f1', false, false)

Running bag-of-words model to learn f1	


In [63]:
model:forward({streams, })